<a href="https://colab.research.google.com/github/duttagoutam/QuantumComputing/blob/main/Model_Interpretability_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Shapash in Jupyter - Overview

<b>With this tutorial you:</b><br />
Understand how Shapash works in Jupyter Notebook
with a simple use case<br />

Contents:
- Build a Regressor
- Compile Shapash SmartExplainer
- Display global and local explanability
- Export local summarized explainability with to_pandas method
- Save Shapash object in pickle file

Data from Kaggle [House Prices](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

In [ ]:
!pip install shapash

     |████████████████████████████████| 873 kB 15.2 MB/s 
     |████████████████████████████████| 75 kB 3.6 MB/s 
     |████████████████████████████████| 1.0 MB 45.2 MB/s 
     |████████████████████████████████| 188 kB 49.1 MB/s 
     |████████████████████████████████| 3.5 MB 49.8 MB/s 
     |████████████████████████████████| 1.8 MB 43.2 MB/s 
     |████████████████████████████████| 13.1 MB 45 kB/s 
     |████████████████████████████████| 642 kB 46.6 MB/s 
     |████████████████████████████████| 3.4 MB 45.3 MB/s 
     |████████████████████████████████| 356 kB 59.5 MB/s 
     |████████████████████████████████| 99 kB 8.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 25.3 MB 80 kB/s 
     |████████████████████████████████| 357 kB 48.9 MB/s 
  Created wheel for dash: filename=dash-1.17.0-py3-none-any.whl size=83737 sha256=ce2fb9ec2affe56c5f3d517569da63372f0ddce6aae4

In [ ]:
!pip install category_encoders

     |████████████████████████████████| 80 kB 5.9 MB/s 


In [ ]:
import pandas as pd
from category_encoders import OrdinalEncoder
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Building Supervized Model 

In [ ]:
from shapash.data.data_loader import data_loading
house_df, house_dict = data_loading('house_prices')

In [ ]:
y_df=house_df['SalePrice'].to_frame()
X_df=house_df[house_df.columns.difference(['SalePrice'])]

In [ ]:
house_df.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2-Story 1946 & Newer,Residential Low Density,8450,Paved,Regular,Near Flat/Level,"All public Utilities (E,G,W,& S)",Inside lot,Gentle slope,College Creek,Normal,Normal,Single-family Detached,Two story,7,5,2003,2003,Gable,Standard (Composite) Shingle,Vinyl Siding,Vinyl Siding,Brick Face,196.0,Good,Average/Typical,Poured Contrete,Good (90-99 inches),Typical - slight dampness allowed,No Exposure/No Basement,Good Living Quarters,706,Unfinished/No Basement,0,150,856,Gas forced warm air furnace,Excellent,Yes,Standard Circuit Breakers & Romex,856,854,0,1710,1,0,2,1,3,1,Good,8,Typical Functionality,0,Attached to home,2003.0,Rough Finished,548,Typical/Average,Typical/Average,Paved,0,61,0,0,0,0,0,2,2008,Warranty Deed - Conventional,Normal Sale,208500
2,1-Story 1946 & Newer All Styles,Residential Low Density,9600,Paved,Regular,Near Flat/Level,"All public Utilities (E,G,W,& S)",Frontage on 2 sides of property,Gentle slope,Veenker,Adjacent to feeder street,Normal,Single-family Detached,One story,6,8,1976,1976,Gable,Standard (Composite) Shingle,Metal Siding,Metal Siding,None,0.0,Average/Typical,Average/Typical,Cinder Block,Good (90-99 inches),Typical - slight dampness allowed,Good Exposure,Average Living Quarters,978,Unfinished/No Basement,0,284,1262,Gas forced warm air furnace,Excellent,Yes,Standard Circuit Breakers & Romex,1262,0,0,1262,0,1,2,0,3,1,Typical/Average,6,Typical Functionality,1,Attached to home,1976.0,Rough Finished,460,Typical/Average,Typical/Average,Paved,298,0,0,0,0,0,0,5,2007,Warranty Deed - Conventional,Normal Sale,181500
3,2-Story 1946 & Newer,Residential Low Density,11250,Paved,Slightly irregular,Near Flat/Level,"All public Utilities (E,G,W,& S)",Inside lot,Gentle slope,College Creek,Normal,Normal,Single-family Detached,Two story,7,5,2001,2002,Gable,Standard (Composite) Shingle,Vinyl Siding,Vinyl Siding,Brick Face,162.0,Good,Average/Typical,Poured Contrete,Good (90-99 inches),Typical - slight dampness allowed,Mimimum Exposure,Good Living Quarters,486,Unfinished/No Basement,0,434,920,Gas forced warm air furnace,Excellent,Yes,Standard Circuit Breakers & Romex,920,866,0,1786,1,0,2,1,3,1,Good,6,Typical Functionality,1,Attached to home,2001.0,Rough Finished,608,Typical/Average,Typical/Average,Paved,0,42,0,0,0,0,0,9,2008,Warranty Deed - Conventional,Normal Sale,223500
4,2-Story 1945 & Older,Residential Low Density,9550,Paved,Slightly irregular,Near Flat/Level,"All public Utilities (E,G,W,& S)",Corner lot,Gentle slope,Crawford,Normal,Normal,Single-family Detached,Two story,7,5,1915,1970,Gable,Standard (Composite) Shingle,Wood Siding,Wood Shingles,None,0.0,Average/Typical,Average/Typical,Brick & Tile,Typical (80-89 inches),Good,No Exposure/No Basement,Average Living Quarters,216,Unfinished/No Basement,0,540,756,Gas forced warm air furnace,Good,Yes,Standard Circuit Breakers & Romex,961,756,0,1717,1,0,1,0,3,1,Good,7,Typical Functionality,1,Detached from home,1998.0,Unfinished/No Garage,642,Typical/Average,Typical/Average,Paved,0,35,272,0,0,0,0,2,2006,Warranty Deed - Conventional,Abnormal Sale,140000
5,2-Story 1946 & Newer,Residential Low Density,14260,Paved,Slightly irregular,Near F

#### Encoding Categorical Features 

In [ ]:
from category_encoders import OrdinalEncoder

categorical_features = [col for col in X_df.columns if X_df[col].dtype == 'object']

encoder = OrdinalEncoder(
    cols=categorical_features,
    handle_unknown='ignore',
    return_df=True).fit(X_df)

X_df=encoder.transform(X_df)

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [ ]:
X_df.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,BsmtFinType2,BsmtFullBath,BsmtHalfBath,BsmtQual,BsmtUnfSF,CentralAir,Condition1,Condition2,Electrical,EnclosedPorch,ExterCond,ExterQual,Exterior1st,Exterior2nd,Fireplaces,Foundation,FullBath,Functional,GarageArea,GarageCond,GarageFinish,GarageQual,GarageType,GarageYrBlt,GrLivArea,HalfBath,Heating,HeatingQC,HouseStyle,KitchenAbvGr,KitchenQual,LandContour,LandSlope,LotArea,LotConfig,LotShape,LowQualFinSF,MSSubClass,MSZoning,MasVnrArea,MasVnrType,MiscVal,MoSold,Neighborhood,OpenPorchSF,OverallCond,OverallQual,PavedDrive,PoolArea,RoofMatl,RoofStyle,SaleCondition,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,856,854,0,3,1,1,1,706,0,1,1,1,0,1,150,1,1,1,1,0,1,1,1,1,0,1,2,1,548,1,1,1,1,2003.0,1710,1,1,1,1,1,1,1,1,8450,1,1,0,1,1,196.0,1,0,2,1,61,5,7,1,0,1,1,1,1,0,1,8,856,1,0,2003,2003,2008
2,1262,0,0,3,1,1,2,978,0,2,1,0,1,1,284,1,2,1,1,0,1,2,2,2,1,2,2,1,460,1,1,1,1,1976.0,1262,0,1,1,2,1,2,1,1,9600,2,1,0,2,1,0.0,2,0,5,2,0,8,6,1,0,1,1,1,1,0,1,6,1262,1,298,1976,1976,2007
3,920,866,0,3,1,1,3,486,0,1,1,1,0,1,434,1,1,1,1,0,1,1,1,1,1,1,2,1,608,1,1,1,1,2001.0,1786,1,1,1,1,1,1,1,1,11250,1,2,0,1,1,162.0,1,0,9,1,42,5,7,1,0,1,1,1,1,0,1,6,920,1,0,2001,2002,2008
4,961,756,0,3,1,2,1,216,0,2,1,1,0,2,540,1,1,1,1,272,1,2,3,3,1,3,1,1,642,1,2,1,2,1998.0,1717,0,1,2,1,1,1,1,1,9550,3,2,0,3,1,0.0,2,0,2,3,35,5,7,1,0,1,1,2,1,0,1,7,756,1,0,1915,1970,2006
5,1145,1053,0,4,1,1,4,655,0,1,1,1,0,1,490,1,1,1,1,0,1,1,1,1,1,1,2,1,836,1,1,1,1,2000.0,2198,1,1,1,1,1,1,1,1,14260,2,2,0,1,1,350.0,1,0,12,4,84,5,8,1,0,1,1,1,1,0,1,9,1145,1,192,2000,2000,2008


#### Train / Test Split

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X_df, y_df, train_size=0.75, random_state=1)

#### Model Fitting

In [ ]:
regressor = LGBMRegressor(n_estimators=200).fit(Xtrain,ytrain)

In [ ]:
y_pred = pd.DataFrame(regressor.predict(Xtest),columns=['pred'],index=Xtest.index)

## Understand my model with shapash

#### Declare and Compile SmartExplainer 

In [ ]:
from shapash.explainer.smart_explainer import SmartExplainer

In [ ]:
house_dict

{'1stFlrSF': 'First Floor square feet',
 '2ndFlrSF': 'Second floor square feet',
 '3SsnPorch': 'Three season porch area in square feet',
 'BedroomAbvGr': 'Bedrooms above grade',
 'BldgType': 'Type of dwelling',
 'BsmtCond': 'General condition of the basement',
 'BsmtExposure': 'Refers to walkout or garden level walls',
 'BsmtFinSF1': 'Type 1 finished square feet',
 'BsmtFinSF2': 'Type 2 finished square feet',
 'BsmtFinType1': 'Rating of basement finished area',
 'BsmtFinType2': 'Rating of basement finished area (if present)',
 'BsmtFullBath': 'Basement full bathrooms',
 'BsmtHalfBath': 'Basement half bathrooms',
 'BsmtQual': 'Height of the basement',
 'BsmtUnfSF': 'Unfinished square feet of basement area',
 'CentralAir': 'Central air conditioning',
 'Condition1': 'Proximity to various conditions',
 'Condition2': 'Proximity to other various conditions',
 'Electrical': 'Electrical system',
 'EnclosedPorch': 'Enclosed porch area in square feet',
 'ExterCond': "Exterior materials' conditio

In [ ]:
xpl = SmartExplainer(features_dict=house_dict) # Optional parameter, dict specifies label for features name 

In [ ]:
xpl.compile(
    x=Xtest,
    model=regressor,
    preprocessing=encoder, # Optional: compile step can use inverse_transform method
    y_pred=y_pred # Optional
)

Backend: Shap TreeExplainer


#### Display features importance

In [ ]:
xpl.plot.features_importance()

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


#### Focus on a specific subset

You can use the features_importance method to compare the contribution of features of a subset to the global features importance

In [ ]:
subset = [ 168, 54, 995, 799, 310, 322, 1374,
          1106, 232, 645, 1170, 1229, 703, 66,  
          886, 160, 191, 1183, 1037, 991, 482,  
          725, 410, 59, 28, 719, 337, 36]
xpl.plot.features_importance(selection=subset)

#### Understand how a feature contributes

- The contribution_plot allows to analyse how one feature affects prediction
- Type of plot depends on the type of features
- You can use feature name, feature label or feature number to specify which feature you want to analyze

In [ ]:
xpl.plot.contribution_plot("OverallQual")

In [ ]:
xpl.plot.contribution_plot("Second floor square feet")

## Display a Summarized but Explicit local explainability

#### Filter method

Use the filter method to specify how to summarize local explainability <br />
There are 4 parameters to customize the summary: <br />
- max_contrib : maximum number of criteria to display
- threshold : minimum value of the contribution (in absolute value) necessary to display a criterion
- positive : display only positive contribution? Negative?(default None)
- features_to_hide : list of features you don't want to display

In [ ]:
xpl.filter(max_contrib=8,threshold=100)

#### Display local plot, applying your filter

you can use row_num, index or query parameter to specify which prediction you want to explain

In [ ]:
xpl.plot.local_plot(index=560)

## Save your Explainer & Export results

#### Export your local explanation to pandas DataFrame:

to_pandas method has the same parameters as the filter method

In [ ]:
summary_df= xpl.to_pandas(
    max_contrib=3, # Number Max of features to show in summary
    threshold=5000,
)

In [ ]:
summary_df.head()

,pred,feature_1,value_1,contribution_1,feature_2,value_2,contribution_2,feature_3,value_3,contribution_3
259,209141.256921,Ground living area square feet,1792,13710.4,Overall material and finish of the house,7,12776.3,Total square feet of basement area,963,-5103.03
268,178734.474531,Ground living area square feet,2192,29747,Overall material and finish of the house,5,-26151.3,Overall condition of the house,8,9190.84
289,113950.844570,Overall material and finish of the house,5,-24730,Ground living area square feet,900,-16342.6,Total square feet of basement area,882,-5922.64
650,74957.162142,Overall material and finish of the house,4,-33927.7,Ground living area square feet,630,-23234.4,Total square feet of basement area,630,-11687.9
1234,135305.243500,Overall material and finish of the house,5,-25445.7,Ground living area square feet,1188,-11476.6,Condition of sale,Abnormal Sale,-5071.82


#### Save your explainer in Pickle File

You can save the SmartExplainer Object in a pickle file to make new plots later or launch the WebApp again

In [ ]:
xpl.save('./xpl.pkl')

In [ ]:
app = xpl.run_app(title_story='House Prices')

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:1490: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.

INFO:root:Your Shapash application run on http://8060323fab5c:8050/


Dash is running on http://0.0.0.0:8050/



INFO:root:Use the method .kill() to down your app.
INFO:shapash.webapp.smart_app:Dash is running on http://0.0.0.0:8050/



 * Serving Flask app "shapash.webapp.smart_app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://0.0.0.0:8050/ (Press CTRL+C to quit)
